In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

#data processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from tensorflow import keras    
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

In [ ]:
data = '../input/7segmentdisplay/sevensegdataset.npy'
features = np.load(data)
X = features[:, :-1] 
Y = features[:, -1]

In [ ]:
labels = Y
data = []
for i in X:
    data.append(np.reshape(i, (28,28)))
data = np.array(data)
#normalize data from 0 to 1
data = np.array(data)/255

In [ ]:
def show_img(arr):
    img = Image.fromarray(np.uint8(arr), 'P')
    plt.imshow(img)
    plt.show()

In [ ]:
def define_model():
    model = Sequential()

    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', input_shape = (28,28,1)))
    model.add(MaxPool2D(pool_size=(2,2)))

    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    
    model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
   

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Dropout(0.4))
    model.add(Dense(10, activation = "softmax"))
    
    opt = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data,labels,test_size=0.2)
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

In [ ]:
label_as_binary = LabelBinarizer()
train__y_labels = label_as_binary.fit_transform(y_train)
test_y_labels =  label_as_binary.fit_transform(y_test)

In [ ]:
model = define_model()
batch_size = 32
epochs = 6

model.fit(x_train, train__y_labels,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, test_y_labels))
score = model.evaluate(x_test, test_y_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save("test_6.h5")
print("Saved model to disk")